In [1]:
import sys
import os
sys.path.append(os.path.abspath('Input_production/source/'))
from osgeo import ogr
import crujra


In [2]:
cru_jra_resolution = 0.5 #degrees

aoi = ogr.Open('Input_production/working/aoi_4326/aoi_4326.shp')
layer = aoi.GetLayerByName('aoi_4326')
extent = layer.GetExtent()

def buffer_extent(extent, buffer, digits=6):
    """ 
    
    parameters
    ----------
    extent: list like
        extent in [minx,maxx,miny,maxy] order
        assumes coords are in degrees
    buffer: float
        buffer in degrees
    digits: int, default 6
        number of digits to pass to round

    returns
    -------
    tuple:
        (minx,maxx,miny,maxy) in degrees
    """
    minx = round(max(extent[0] - buffer, -180.0), 6)
    maxx = round(min(extent[1] + buffer,  180.0), 6)
    
    miny = round(max(extent[2] - buffer, -90.0), 6)
    maxy = round(min(extent[3] + buffer,  90.0), 6)

    
    return (minx, maxx, miny, maxy)
                  

aoi_extent = buffer_extent(extent, cru_jra_resolution+.1) #.1° is ~5km + cru resolution
aoi_extent

/Users/rwspicer/miniconda3/envs/ip_0/lib/python3.11/site-packages/osgeo/ogr.py:601: FutureWarning: Neither ogr.UseExceptions() nor ogr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


(-180.0, 180.0, 44.930151, 84.223125)

In [3]:
_1901 = crujra.CRU_JRA_daily(1901, 'test_data_py', verbose=True, _vars=['tmax','dswrf'], aoi_extent=aoi_extent)

# 1 YEAR FOR ARCTIC 

Loading from raw data at 'test_data_py'
..loading raw data for 'tmax' from 'test_data_py/tmax/crujra.v2.5.5d.tmax.1901.365d.noc.nc.gz'
..clipping to aoi
..loading raw data for 'dswrf' from 'test_data_py/dswrf/crujra.v2.5.5d.dswrf.1901.365d.noc.nc.gz'
..clipping to aoi
..All raw data successfully loaded clipped and resampled.
dataset initialized


In [4]:
_1901.save('test_1901.nc')

In [3]:
_1901_load = crujra.CRU_JRA_daily(1901,'test_1901.nc', verbose=True, _vars=['tmax','dswrf'])
# _1901_load.load()

loading file 'test_1901.nc' assuming correct timestemp and region are set
dataset initialized


In [4]:
_1901_load.dataset#.coords

<xarray.Dataset> Size: 164MB
Dimensions:  (time: 365, lat: 78, lon: 720)
Coordinates:
  * lon      (lon) float64 6kB -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float64 624B 45.25 45.75 46.25 46.75 ... 82.75 83.25 83.75
  * time     (time) object 3kB 1901-01-01 00:00:00 ... 1901-12-31 00:00:00
Data variables:
    tmax     (time, lat, lon) float32 82MB ...
    dswrf    (time, lat, lon) float32 82MB ...
Attributes:
    CDI:              Climate Data Interface version 1.9.9 (https://mpimet.mp...
    Conventions:      CF-1.4
    source:           Data is provided from the Japanese 55-year Reanalysis (...
    institution:      Produced at the Climatic Research Unit, UEA, Norwich UK
    title:            CRUJRA TMAX: a forcing dataset based on CRU TS v4.08 an...
    version_control:  V2.5: Update to 2026 with partial alignment to CRU TS v...
    history:          Thu Jun 27 20:44:01 2024: cdo -L -s copy /media/f098/bi...
    references:       Contact i.harris@uea.ac.uk with any questions
    comment:          A copy of JRA-55 year 1958 regridded to CRU 0.5 grid an...
    contact:          i.harris@uea.ac.uk
    CDO:              Climate Data Operators version 1.9.9 (https://mpimet.mp...

True

In [19]:
import xarray as xr
import numpy as np

In [125]:
ds0 = xr.open_dataset('test-wnc.nc', engine='netcdf4')
ds0

<xarray.Dataset> Size: 17MB
Dimensions:                       (y: 1934, x: 2242)
Coordinates:
  * x                             (x) float64 18kB -4.6e+06 ... 4.364e+06
  * y                             (y) float64 15kB 4.249e+06 ... -3.483e+06
Data variables:
    Band1                         (y, x) float32 17MB ...
    lambert_azimuthal_equal_area  |S1 1B ...
Attributes:
    Conventions:         CF-1.5
    GDAL:                GDAL 3.10.0, released 2024/11/01
    history:             Wed Jan 29 14:44:47 2025: GDAL Create( test-wnc.nc, ...
    GDAL_AREA_OR_POINT:  Area

In [60]:
ds = xr.open_dataset("/Users/rwspicer/Desktop/work/TEM/Input_production/working/aoi_5km_buffer_6931.tiff", engine="rasterio")

In [ ]:
np.

In [69]:
ds.band

<xarray.DataArray 'band' (band: 1)> Size: 8B
array([1])
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...

In [62]:
minx=-4602000.0
maxx=4366000.0
x_res=4000
miny= 4251000.0
maxy=-3485000.0
y_res=-4000
lat_dim = np.arange(miny,maxy, abs(y_res)) + (abs(y_res)/2)
if maxy < miny:
    miny, maxy = maxy, miny
    lat_dim = np.arange(miny,maxy, abs(y_res)) + (abs(y_res)/2)
    lat_dim = lat_dim[::-1]
    miny, maxy = maxy, miny
lon_dim = np.arange(minx,maxx, abs(x_res)) + (abs(x_res)/2)

In [126]:
ds2=xr.open_dataset('test_wc.nc')
ds2.tmin

<xarray.DataArray 'tmin' (lat: 1934, lon: 2242)> Size: 17MB
[4336028 values with dtype=float32]
Coordinates:
    lon      (lat) float64 15kB ...
  * lat      (lat) float64 15kB 4.249e+06 4.245e+06 ... -3.479e+06 -3.483e+06

In [108]:

xr.DataArray(np.ones(2242* 1934).reshape([2242, 1934]), dims=['lat', 'lon'])


<xarray.DataArray (lat: 2242, lon: 1934)> Size: 35MB
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], shape=(2242, 1934))
Dimensions without coordinates: lat, lon